In [32]:
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, Polygon, LineString

from tqdm.auto import tqdm

In [33]:
trees = gpd.read_file('../results/yerevan_trees.geojson')

In [34]:
buildings = gpd.read_file('../data/yerevan_buildings_clean.geojson')

In [35]:
districts = gpd.read_file('../data/yerevan_districts.geojson')

In [26]:
## FILTER FOR DISTRICT

trees = trees.sjoin(districts[districts['name'] == 'Kentron'][['geometry']], predicate='intersects').reset_index(drop=True)
trees.drop(columns=['index_right'], inplace=True)

buildings = buildings.sjoin(districts[districts['name'] == 'Kentron'][['geometry']], predicate='within')
buildings.drop(columns=['index_right'], inplace=True)

buildings = buildings[['building_id', 'img_id', 'name', 'geometry']].reset_index(drop=True)

In [36]:
trees

,img_id,geometry
0,100,POINT (44.56453 40.09380)
1,100,POINT (44.56661 40.09379)
2,100,POINT (44.56503 40.09376)
3,100,POINT (44.56513 40.09374)
4,100,POINT (44.56703 40.09376)
...,...,...
267916,998,POINT (44.49861 40.13505)
267917,998,POINT (44.49810 40.13504)
267918,998,POINT (44.49836 40.13504)
267919,998,POINT (44.49864 40.13504)


In [38]:
def count_trees(trees_gdf, buildings_gdf):
    """
    Tags each tree with building_id(s) it is visible from based on circular visibility
    from the centroid of the building with a radius equal to the building's size (length/width),
    considering line-of-sight obstruction by other buildings within 500m.

    Parameters:
    - trees_gdf: GeoDataFrame of tree Points (EPSG:4326)
    - buildings_gdf: GeoDataFrame of building Polygons with a 'building_id' column (EPSG:4326)

    Returns:
    - buildings_with_counts: GeoDataFrame of buildings with 'visible_trees' count
    - visible_trees_with_building_ids: GeoDataFrame of all trees with 'building_id' (can be None or multiple)
    """

    # Project to metric CRS for accurate distance measurements
    buildings_proj = buildings_gdf.to_crs(epsg=3857).copy()
    trees_proj = trees_gdf.to_crs(epsg=3857).copy()

    buildings_proj["visible_trees"] = 0
    tree_visibility = {i: [] for i in trees_proj.index}

    for _, building in tqdm(buildings_proj.iterrows(), total=len(buildings_proj)):
        b_id = building["building_id"]
        building_geom = building.geometry
        centroid = building_geom.centroid

        # Buffer radius for visibility (max of building size or 100m)
        minx, miny, maxx, maxy = building_geom.bounds
        buffer_radius = max(maxx - minx, maxy - miny, 100)
        visibility_zone = centroid.buffer(buffer_radius)

        # Trees within visibility range
        nearby_trees = trees_proj[trees_proj.geometry.within(visibility_zone)]

        # Buildings within 500m of this building to check for obstruction
        obstruction_zone = centroid.buffer(500)
        potential_blockers = buildings_proj[
            (buildings_proj["building_id"] != b_id) &
            (buildings_proj.geometry.intersects(obstruction_zone))
        ]

        visible_tree_indices = set()

        for _, tree in nearby_trees.iterrows():
            tree_point = tree.geometry
            line_of_sight = LineString([centroid, tree_point])

            # Check if any nearby building blocks the view
            blocked = potential_blockers.geometry.intersects(line_of_sight).any()

            if not blocked:
                tree_visibility[tree.name].append(b_id)
                visible_tree_indices.add(tree.name)

        buildings_proj.loc[buildings_proj["building_id"] == b_id, "visible_trees"] = len(visible_tree_indices)

    # Construct output GeoDataFrame for trees with associated building IDs
    tree_records = []
    for i, row in trees_proj.iterrows():
        building_ids = tree_visibility[i]
        if not building_ids:
            tree_records.append({"building_id": None, "geometry": row.geometry})
        else:
            for b_id in building_ids:
                tree_records.append({"building_id": b_id, "geometry": row.geometry})

    visible_trees_gdf = gpd.GeoDataFrame(tree_records, geometry="geometry", crs=trees_proj.crs)

    return buildings_proj.to_crs(epsg=4326), visible_trees_gdf.to_crs(epsg=4326)

In [ ]:
# def count_trees(trees_gdf, buildings_gdf):
#     """
#     Tags each tree with building_id(s) it is visible from based on circular visibility
#     from the centroid of the building with a radius equal to the building's size (length/width).

#     Parameters:
#     - trees_gdf: GeoDataFrame of tree Points (EPSG:4326)
#     - buildings_gdf: GeoDataFrame of building Polygons with a 'building_id' column (EPSG:4326)

#     Returns:
#     - buildings_with_counts: GeoDataFrame of buildings with 'visible_trees' count
#     - visible_trees_with_building_ids: GeoDataFrame of all trees with 'building_id' (can be None or multiple)
#     """
    
#     # Project to metric CRS (for accurate distance measurements)
#     buildings_proj = buildings_gdf.to_crs(epsg=3857).copy()
#     trees_proj = trees_gdf.to_crs(epsg=3857).copy()

#     buildings_proj["visible_trees"] = 0
#     tree_visibility = {i: [] for i in trees_proj.index}  # collect all visible building_ids per tree

#     for _, building in tqdm(buildings_proj.iterrows(), total=len(buildings_proj)):
#         b_id = building["building_id"]
#         building_geom = building.geometry

#         # Calculate centroid of the building
#         centroid = building_geom.centroid

#         # Calculate the building's length and width (bbox)
#         minx, miny, maxx, maxy = building_geom.bounds
#         length = maxx - minx
#         width = maxy - miny
        
#         # Set the buffer radius to the maximum of the length and width of the building
#         buffer_radius = max(length, width)

#         # Create a circular buffer around the centroid
#         buffer_zone = centroid.buffer(max(buffer_radius, 100))

#         # Find trees within the buffer zone
#         nearby_trees = trees_proj[trees_proj.geometry.within(buffer_zone)]

#         # Collect visible trees' indices for this building
#         for _, tree in nearby_trees.iterrows():
#             tree_visibility[tree.name].append(b_id)

#         # Count the number of visible trees for this building
#         visible_tree_indices = set()
#         for _, tree in nearby_trees.iterrows():
#             visible_tree_indices.add(tree.name)
        
#         buildings_proj.loc[buildings_proj["building_id"] == b_id, "visible_trees"] = len(visible_tree_indices)

#     # Build the output GeoDataFrame for trees with visible building IDs
#     tree_records = []
#     for i, row in trees_proj.iterrows():
#         building_ids = tree_visibility[i]
#         if not building_ids:
#             tree_records.append({"building_id": None, "geometry": row.geometry})
#         else:
#             for b_id in building_ids:
#                 tree_records.append({"building_id": b_id, "geometry": row.geometry})

#     visible_trees_gdf = gpd.GeoDataFrame(tree_records, geometry="geometry", crs=trees_proj.crs)

#     # Return to original CRS
#     return buildings_proj.to_crs(epsg=4326), visible_trees_gdf.to_crs(epsg=4326)

In [39]:
buildings_with_counts, visible_trees_with_building_ids = count_trees(trees, buildings)

  0%|          | 0/76205 [00:00<?, ?it/s]

In [40]:
visible_trees_with_building_ids

,building_id,geometry
0,NaN,POINT (44.56453 40.09380)
1,NaN,POINT (44.56661 40.09379)
2,NaN,POINT (44.56503 40.09376)
3,NaN,POINT (44.56513 40.09374)
4,NaN,POINT (44.56703 40.09376)
...,...,...
953461,NaN,POINT (44.49810 40.13504)
953462,NaN,POINT (44.49836 40.13504)
953463,NaN,POINT (44.49864 40.13504)
953464,82782.0,POINT (44.49991 40.13503)


In [41]:
buildings_with_counts

,osmid,name,name:en,building,addr:country,addr:city,addr:district,addr:region,addr:housenumber,addr:postcode,addr:street,check_date,img_id,building_id,geometry,visible_trees
0,218017460,None,None,yes,AM,Երևան,None,None,96,None,Շիրակի փողոց,None,1009,0,"POLYGON ((44.43474 40.13809, 44.43491 40.13746...",0
1,542793077,None,None,yes,AM,Երևան,None,None,None,None,None,None,1009,1,"POLYGON ((44.43557 40.13686, 44.43589 40.13663...",0
2,542793078,None,None,yes,AM,Երևան,None,None,None,None,None,None,1009,2,"POLYGON ((44.43549 40.13656, 44.43557 40.13618...",0
3,577406990,None,None,yes,AM,Երևան,None,None,None,None,None,None,1009,3,"POLYGON ((44.43620 40.13674, 44.43663 40.13644...",0
4,577407003,None,None,yes,AM,Երևան,None,None,None,None,None,None,1009,4,"POLYGON ((44.43389 40.13587, 44.43389 40.13539...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76200,1229748417,None,None,yes,None,None,None,None,None,None,None,None,998,82830,"POLYGON ((44.49528 40.13706, 44.49529 40.13729...",23
76201,1229748418,None,None,yes,None,None,None,None,None,None,None,None,998,82831,"POLYGON ((44.49426 40.13564, 44.49441 40.13564...",38
76202,1229748419,None,None,yes,None,None,None,None,None,None,None,None,998,82832,"POLYGON ((44.49512 40.13600, 44.49497 40.13600...",23
76203,1229748420,None,None,yes,None,None,None,None,None,None,None,None,998,82833,"POLYGON ((44.49495 40.13594, 44.49496 40.13607...",19


In [42]:
visible_trees_with_building_ids.to_file('../data/trees_with_buildins_ids_no_blocking_view.geojson')

In [43]:
buildings_with_counts[['osmid', 'building_id', 'img_id', 'visible_trees', 'geometry']].to_file('../data/buildings_with_tree_counts_no_blocking_view.geojson')